In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os
import time
import json
import requests
from copy import deepcopy
from dotenv import load_dotenv
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Reading GeoJSON files

In [2]:
with open(r"/home/mohammed/Desktop/Musaddiq/Coursera_Capstone/json/cities.json", 'r', encoding='utf8', errors='ignore') as file:
    cities = json.load(file)
    
with open(r"/home/mohammed/Desktop/Musaddiq/Coursera_Capstone/json/districts.json", 'r', encoding='utf8', errors='ignore') as file:
    districts = json.load(file)

Getting city_id of Al Khobar and Dammam

In [3]:
for city in cities:
    if "Khobar" in city["name_en"]:
        print(str(city["city_id"]) + " " + city["name_en"])
    if "Dammam" in city["name_en"]:
        print(str(city["city_id"]) + " " + city["name_en"])

13 Dammam
31 Al Khobar


Getting districts in Al Khobar and Dammam

In [4]:
khobar_districts = []
dammam_districts = []

for district in districts:
    if district["city_id"] == 31:
        khobar_districts.append(district)
        
for district in districts:
    if district["city_id"] == 13:
        dammam_districts.append(district)

Creating a copy in XY Coordinates for GeoJSON

In [5]:
khobar_districts_xy = deepcopy(khobar_districts)
dammam_districts_xy = deepcopy(dammam_districts)

Reversing boundaries coordinates as per GeoJSON format

In [6]:
for district in range(len(khobar_districts_xy)):
    for _ in khobar_districts_xy[district]["boundaries"][0]:
        _.reverse()
        
for district in range(len(dammam_districts_xy)):
    for _ in dammam_districts_xy[district]["boundaries"][0]:
        _.reverse()
        
khobar_districts

[{'district_id': 10500031001,
  'city_id': 31,
  'region_id': 5,
  'name_ar': 'حي التحلية',
  'name_en': 'At Tahliyah Dist.',
  'boundaries': [[[26.18209534, 50.21638279],
    [26.18118287, 50.21611424],
    [26.18140407, 50.2156978],
    [26.17970601, 50.21392926],
    [26.17971959, 50.21352359],
    [26.17803992, 50.2118197],
    [26.17732234, 50.21144898],
    [26.17361962, 50.20759408],
    [26.17277879, 50.20861849],
    [26.17056918, 50.2061992],
    [26.17123995, 50.20544797],
    [26.17086068, 50.2046207],
    [26.16648087, 50.20121919],
    [26.16439652, 50.20023882],
    [26.16228008, 50.19795616],
    [26.16000799, 50.19650694],
    [26.15941666, 50.19610723],
    [26.15888672, 50.19574902],
    [26.15854372, 50.19551717],
    [26.15837159, 50.19540082],
    [26.15803023, 50.19518369],
    [26.15737514, 50.19472729],
    [26.15631941, 50.19332988],
    [26.15601034, 50.1929208],
    [26.15526795, 50.19193817],
    [26.15451659, 50.1909437],
    [26.15416384, 50.1906981],
   

In [7]:
for district in range(len(khobar_districts_xy)):
    khobar_districts_xy[district]["center"] = [sum(x)/len(x) for x in zip(*khobar_districts_xy[district]["boundaries"][0])]
    khobar_districts_xy[district]["latitude"] = khobar_districts_xy[district]["center"][0]
    khobar_districts_xy[district]["longitude"] = khobar_districts_xy[district]["center"][1]    
    
for district in range(len(khobar_districts)):
    khobar_districts[district]["center"] = [sum(x)/len(x) for x in zip(*khobar_districts[district]["boundaries"][0])]
    khobar_districts[district]["latitude"] = khobar_districts[district]["center"][0]
    khobar_districts[district]["longitude"] = khobar_districts[district]["center"][1]
    
for district in range(len(dammam_districts_xy)):
    dammam_districts_xy[district]["center"] = [sum(x)/len(x) for x in zip(*dammam_districts_xy[district]["boundaries"][0])]
    dammam_districts_xy[district]["latitude"] = dammam_districts_xy[district]["center"][0]
    dammam_districts_xy[district]["longitude"] = dammam_districts_xy[district]["center"][1]
    
for district in range(len(dammam_districts)):
    dammam_districts[district]["center"] = [sum(x)/len(x) for x in zip(*dammam_districts[district]["boundaries"][0])]
    dammam_districts[district]["latitude"] = dammam_districts[district]["center"][0]
    dammam_districts[district]["longitude"] = dammam_districts[district]["center"][1]

In [8]:
khobar = pd.DataFrame(khobar_districts)
dammam = pd.DataFrame(dammam_districts)

khobar_xy = pd.DataFrame(khobar_districts_xy)
dammam_xy = pd.DataFrame(dammam_districts_xy)


khobar_xy

,district_id,city_id,region_id,name_ar,name_en,boundaries,center,latitude,longitude
0,10500031001,31,5,حي التحلية,At Tahliyah Dist.,"[[[50.21638279, 26.18209534], [50.21611424, 26...","[50.19358769394619, 26.17781863856502]",50.193588,26.177819
1,10500031002,31,5,حي ابن سيناء,Ibn Sina Dist.,"[[[50.21139022, 26.2507773], [50.21288113, 26....","[50.201773003593736, 26.241028340781252]",50.201773,26.241028
2,10500031003,31,5,حي الحزام الاخضر,Al Hizam Al Akhdar Dist.,"[[[50.19060022, 26.3089098], [50.19145503, 26....","[50.20295822975609, 26.305178597317074]",50.202958,26.305179
3,10500031004,31,5,حي صناعية الثقبة,Sinaiyah Ath Thuqbah Dist.,"[[[50.19749806, 26.24944191], [50.19719675, 26...","[50.19638499861113, 26.253792497222218]",50.196385,26.253792
4,10500031005,31,5,حي التعاون,At Taawun Dist.,"[[[50.19458339, 26.23354342], [50.1963495, 26....","[50.18608636999999, 26.226340009404773]",50.186086,26.226340
5,10500031006,31,5,حي الراكة الجنوبية,Ar Rakah Al Janubiyah Dist.,"[[[50.20035662, 26.33674744], [50.19986525, 26...","[50.203745644269645, 26.350680400337087]",50.203746,26.350680
6,10500031007,31,5,حي الخبر الشمالية,Al Khubar Ash Shamaliyah Dist.,"[[[50.21811139, 26.28045052], [50.21783621, 26...","[50.214117997064186, 26.29401644211008]",50.214118,26.294016
7,10500031008,31,5,حي مدينة العمال,Madinat Al Ummal Dist.,"[[[50.20487945, 26.28181475], [50.20483605, 26...","[50.205489442898546, 26.294228553913037]",50.205489,26.294229
8,10500031009,31,5,حي العقربية,Al Aqrabiyah Dist.,"[[[50.19759817, 26.28393879], [50.19757992, 26...","[50.189789413188414, 26.297885457971017]",50.189789,26.297885
9,10500031010,31,5,حي الخبر الجنوبية,Al Khubar Al Janubiyah Dist.,"[[[50.21598118, 26.26997671], [50.21521935, 26...","[50.20615011927536, 26.27328286536233]",50.206150,26.273283


In [9]:
features = []

for district in range(len(khobar_districts_xy)):
    feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": khobar_districts_xy[district]["boundaries"]},
            "properties": {
                "district_id": khobar_districts_xy[district]["district_id"],
                "city_id": khobar_districts_xy[district]["city_id"],
                "name_en": khobar_districts_xy[district]["name_en"]}
        }

    features.append(feature)
    
khobar_geojson = {
    "type": "FeatureCollection",
    "features": features
}

khobar_geojson = json.dumps(khobar_geojson)
khobar_geojson

'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[50.21638279, 26.18209534], [50.21611424, 26.18118287], [50.2156978, 26.18140407], [50.21392926, 26.17970601], [50.21352359, 26.17971959], [50.2118197, 26.17803992], [50.21144898, 26.17732234], [50.20759408, 26.17361962], [50.20861849, 26.17277879], [50.2061992, 26.17056918], [50.20544797, 26.17123995], [50.2046207, 26.17086068], [50.20121919, 26.16648087], [50.20023882, 26.16439652], [50.19795616, 26.16228008], [50.19650694, 26.16000799], [50.19610723, 26.15941666], [50.19574902, 26.15888672], [50.19551717, 26.15854372], [50.19540082, 26.15837159], [50.19518369, 26.15803023], [50.19472729, 26.15737514], [50.19332988, 26.15631941], [50.1929208, 26.15601034], [50.19193817, 26.15526795], [50.1909437, 26.15451659], [50.1906981, 26.15416384], [50.19041048, 26.15375074], [50.18975249, 26.15280568], [50.18840659, 26.15150019], [50.18791179, 26.15145278], [50.18722404, 26.15142866]

In [10]:
khobar_data = khobar[["name_en", "district_id", "center"]]
dammam_data = dammam[["name_en", "district_id", "center"]]
dammam_data['randNumCol'] = np.random.randint(1, 100, dammam_data.shape[0])
dammam_data

<ipython-input-10-1d4f6625bfcd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dammam_data['randNumCol'] = np.random.randint(1, 100, dammam_data.shape[0])


,name_en,district_id,center,randNumCol
0,Al Khalidiyah Ash Shamaliyah Dist.,10500013001,"[26.413035744059407, 50.14636082148513]",45
1,An Nasriyah Dist.,10500013002,"[26.424016825090902, 50.12204671218182]",83
2,1St Industrial Dist.,10500013003,"[26.39701435782051, 50.14309465038462]",60
3,Al Fanar Dist.,10500013004,"[26.40588090352941, 50.19358098470589]",39
4,Al Athir Dist.,10500013005,"[26.434276787415733, 50.06036651483147]",10
...,...,...,...,...
78,Al Fursan Dist.,10500013080,"[26.35272135157895, 49.96080892276316]",36
79,Al Maha Dist.,10500013081,"[26.48522071253113, 49.93339311850622]",40
80,An Nada Dist.,10500013082,"[26.376905117333333, 50.072901341333335]",28
81,3Rd Industrial City,10500013083,"[25.92946262515, 49.94996354885003]",78


In [11]:
# create a plain map
khobar_map = folium.Map(location=[26.2172,50.1971], zoom_start=15)

folium.GeoJson(khobar_geojson).add_to(khobar_map)

# display map
khobar_map

In [12]:
features = []

for district in range(len(dammam_districts_xy)):
    feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": dammam_districts_xy[district]["boundaries"]},
            "properties": {
                "district_id": dammam_districts_xy[district]["district_id"],
                "city_id": dammam_districts_xy[district]["city_id"],
                "name_en": dammam_districts_xy[district]["name_en"]}
        }

    features.append(feature)
    
dammam_geojson = {
    "type": "FeatureCollection",
    "features": features
}

dammam_geojson = json.dumps(dammam_geojson)
dammam_geojson

'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[50.13177952, 26.42026955], [50.13205218, 26.42399558], [50.13260661, 26.43157151], [50.13320911, 26.43118517], [50.13579565, 26.42954304], [50.13773151, 26.42825795], [50.13859498, 26.42774527], [50.13929657, 26.42740346], [50.14018704, 26.42698971], [50.14057188, 26.42680773], [50.14187171, 26.4262229], [50.14247165, 26.42597332], [50.14438751, 26.42510084], [50.14641129, 26.42426434], [50.1489969, 26.42357042], [50.15075953, 26.42308935], [50.15107467, 26.42300335], [50.15139683, 26.42292416], [50.15241867, 26.42265401], [50.15319429, 26.42245241], [50.15319743, 26.42245157], [50.15320505, 26.4224496], [50.15322584, 26.42244419], [50.15335143, 26.42241154], [50.15402469, 26.42223652], [50.15537304, 26.42188602], [50.15645113, 26.42154448], [50.15786851, 26.42112703], [50.15881739, 26.42084454], [50.15944261, 26.4206659], [50.16048973, 26.42042803], [50.16077069, 26.420325

In [13]:
# create a plain map
dammam_map = folium.Map(location=[26.4207,50.0888], zoom_start=15)

#folium.GeoJson(dammam_geojson).add_to(dammam_map)

folium.Choropleth(
    geo_data=dammam_geojson,
    name='Dammam',
    data=dammam_data,
    columns=['district_id', 'randNumCol'],
    key_on='feature.properties.district_id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Musaddiq'
).add_to(dammam_map)

# display map
dammam_map

In [14]:
#using python-dotenv to protect Foursqaure credentials
%load_ext dotenv
%dotenv
import os

CLIENT_ID = os.getenv("CLIENT_ID") # your Foursquare ID
CLIENT_SECRET = os.getenv("CLIENT_SECRET") # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID SIZE: ' + str(len(CLIENT_ID)))
print('CLIENT_SECRET SIZE: ' + str(len(CLIENT_SECRET)))

Your credentails:
CLIENT_ID SIZE: 48
CLIENT_SECRET SIZE: 48


In [15]:
def getNearbyVenues(
    names,
    latitudes,
    longitudes,
    radius=500,
    ):

    venues_list = []
    for (name, lat, lng) in zip(names, latitudes, longitudes):

        # create the API request URL

        url = \
            'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT,
            )

        # make the GET request

        results = requests.get(url).json()['response']['groups'][0]['items']

        # return only relevant information for each nearby venue

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'],
            ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list
                                 for item in venue_list])
    nearby_venues.columns = [
        'District',
        'District Latitude',
        'District Longitude',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category',
        ]

    return nearby_venues


In [16]:
khobar_venues = getNearbyVenues(names=khobar['name_en'],
                                   latitudes=khobar['latitude'],
                                   longitudes=khobar['longitude']
                                  )

khobar_venues

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,At Tahliyah Dist.,26.177819,50.193588,ملعب اسكان تحليه المياه - الخبر,26.178656,50.193112,Soccer Field
1,At Tahliyah Dist.,26.177819,50.193588,حديقة التحليه,26.179251,50.195127,Garden
2,At Tahliyah Dist.,26.177819,50.193588,معجنات الناعورة,26.177397,50.195792,Breakfast Spot
3,At Tahliyah Dist.,26.177819,50.193588,اسكان تحلية المياة - الخبر,26.179223,50.195842,Harbor / Marina
4,Ibn Sina Dist.,26.241028,50.201773,Al-Seef Cafe & Restaurant (مقهى ومطعم السيف),26.245261,50.201800,Hookah Bar
...,...,...,...,...,...,...,...
819,Al Amwaj Dist.,26.143114,50.145837,Alsubaie farm,26.142485,50.142363,Farm
820,As Sadafah Dist.,26.368823,50.210278,CUE NINE billiard Club,26.365770,50.207950,Pool Hall
821,As Sadafah Dist.,26.368823,50.210278,استاد مدينة الأمير سعود بن جلوي الرياضية بالرا...,26.368982,50.205988,Soccer Stadium
822,As Sadafah Dist.,26.368823,50.210278,AFTER 10,26.366003,50.208199,Nightclub


In [17]:
dammam_venues = getNearbyVenues(names=dammam['name_en'],
                                   latitudes=dammam['latitude'],
                                   longitudes=dammam['longitude']
                                  )

dammam_venues

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Al Khalidiyah Ash Shamaliyah Dist.,26.413036,50.146361,ALSAGGAF TRADING CO.,26.413476,50.151339,Furniture / Home Store
1,An Nasriyah Dist.,26.424017,50.122047,Jawsq Festival Hall (قاعة الجوسق للإحتفالات),26.422153,50.118145,Event Space
2,An Nasriyah Dist.,26.424017,50.122047,بوفية السعادة,26.424839,50.118922,Breakfast Spot
3,An Nasriyah Dist.,26.424017,50.122047,حلويات جرير القباني patisserie jareer,26.422789,50.123262,Pastry Shop
4,An Nasriyah Dist.,26.424017,50.122047,Qatif Mall,26.423454,50.121995,Shopping Mall
...,...,...,...,...,...,...,...
1082,An Nada Dist.,26.376905,50.072901,Macdonald’s,26.377904,50.072297,Fast Food Restaurant
1083,An Nada Dist.,26.376905,50.072901,أم الحصم,26.376780,50.070604,Bay
1084,An Nada Dist.,26.376905,50.072901,تموينات القحطاني,26.374203,50.074840,Grocery Store
1085,An Nada Dist.,26.376905,50.072901,حديقة الياسمين,26.373519,50.074853,Park


In [18]:
# one hot encoding
khobar_onehot = pd.get_dummies(khobar_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column back to dataframe
khobar_onehot['District'] = khobar_venues['District'] 

# move District column to the first column
khobar_onehot = khobar_onehot[ ['District'] + [ col for col in khobar_onehot.columns if col != 'District' ] ]



# one hot encoding
dammam_onehot = pd.get_dummies(dammam_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column back to dataframe
dammam_onehot['District'] = dammam_venues['District'] 

# move District column to the first column
dammam_onehot = dammam_onehot[ ['District'] + [ col for col in dammam_onehot.columns if col != 'District' ] ]



khobar_onehot.head()

,District,ATM,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Theme Restaurant,Tourist Information Center,Track,Trail,Turkish Restaurant,Vape Store,Video Game Store,Watch Shop,Waterfront,Yoga Studio
0,At Tahliyah Dist.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,At Tahliyah Dist.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,At Tahliyah Dist.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,At Tahliyah Dist.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ibn Sina Dist.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
khobar_grouped = khobar_onehot.groupby('District').mean().reset_index()
dammam_grouped = dammam_onehot.groupby('District').mean().reset_index()

khobar_grouped

,District,ATM,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Theme Restaurant,Tourist Information Center,Track,Trail,Turkish Restaurant,Vape Store,Video Game Store,Watch Shop,Waterfront,Yoga Studio
0,Al Amwaj Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Al Andalus Dist.,0.000000,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.00,0.000000,...,0.060606,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Al Aqiq Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Al Aqrabiyah Dist.,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000
4,Al Bahar Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Al Bandariyah Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.026316,0.000000,0.000000,0.000000,0.000000,0.078947,0.000000,0.052632
6,Al Buhayrah Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Al Bustan Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.028571,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Al Hada Dist.,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Al Hamra Dist.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
khobar_venues_sorted = pd.DataFrame(columns=columns)
khobar_venues_sorted['District'] = khobar_grouped['District']

for ind in np.arange(khobar_grouped.shape[0]):
    khobar_venues_sorted.iloc[ind, 1:] = return_most_common_venues(khobar_grouped.iloc[ind, :], num_top_venues)

khobar_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Amwaj Dist.,Farm,Yoga Studio,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market
1,Al Andalus Dist.,Coffee Shop,Café,Restaurant,Bakery,Theme Restaurant,Indian Restaurant,Clothing Store,Seafood Restaurant,Butcher,Lebanese Restaurant
2,Al Aqiq Dist.,Resort,Yoga Studio,Event Service,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market,Farm
3,Al Aqrabiyah Dist.,Coffee Shop,Shawarma Place,Breakfast Spot,Bakery,Fast Food Restaurant,Tea Room,Indian Restaurant,BBQ Joint,Pastry Shop,Egyptian Restaurant
4,Al Bahar Dist.,Bay,Ice Cream Shop,Beach,Outdoor Sculpture,Eastern European Restaurant,Egyptian Restaurant,Donut Shop,French Restaurant,Food Truck,Food Service
5,Al Bandariyah Dist.,Coffee Shop,Watch Shop,Yoga Studio,Gym / Fitness Center,Seafood Restaurant,Boutique,Dessert Shop,Track,Furniture / Home Store,Pharmacy
6,Al Buhayrah Dist.,Lake,Trail,Yoga Studio,French Restaurant,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market
7,Al Bustan Dist.,Coffee Shop,Café,Furniture / Home Store,Indian Restaurant,Intersection,Restaurant,Breakfast Spot,Lounge,Electronics Store,Coffee Roaster
8,Al Hada Dist.,Coffee Shop,Burger Joint,Plaza,Kids Store,Asian Restaurant,Juice Bar,Flower Shop,Clothing Store,Mediterranean Restaurant,Park
9,Al Hamra Dist.,Waterfront,Lake,Entertainment Service,Event Service,Shopping Mall,Beach,Hotel,Food Truck,Food & Drink Shop,Home Service


In [22]:
# create a new dataframe
dammam_venues_sorted = pd.DataFrame(columns=columns)
dammam_venues_sorted['District'] = dammam_grouped['District']

for ind in np.arange(dammam_grouped.shape[0]):
    dammam_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dammam_grouped.iloc[ind, :], num_top_venues)

dammam_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1St Industrial Dist.,Furniture / Home Store,Coffee Shop,Construction & Landscaping,Harbor / Marina,Fish & Chips Shop,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store
1,2Nd Industrial City,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Mountain,Fried Chicken Joint,Convenience Store,Donut Shop,Farm,Falafel Restaurant,Event Space
2,Ad Danah Dist.,Restaurant,Middle Eastern Restaurant,Pizza Place,Coffee Shop,Café,Food Court,Falafel Restaurant,Steakhouse,Supermarket,Stationery Store
3,Ad Dawasir Dist.,Jewelry Store,Men's Store,Clothing Store,Market,Shop & Service,Home Service,Shopping Mall,Afghan Restaurant,Arts & Crafts Store,Breakfast Spot
4,Al Adamah Dist.,Coffee Shop,Hotel,Restaurant,Pool Hall,Diner,Cosmetics Shop,Doner Restaurant,Fast Food Restaurant,Farm,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
70,Madinat Al Ummal Dist.,Convenience Store,Coffee Shop,Café,Intersection,Beach,Market,Burger Joint,Middle Eastern Restaurant,Furniture / Home Store,Indian Restaurant
71,Prince Muhammed Ibn Saud Dist.,Café,Breakfast Spot,Gym / Fitness Center,Stadium,Convenience Store,Pakistani Restaurant,Pizza Place,Department Store,Dessert Shop,Seafood Restaurant
72,Qasr Al Khalij Dist.,Middle Eastern Restaurant,Palace,Café,Food Truck,Yemeni Restaurant,Donut Shop,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space
73,Taibah Dist.,Coffee Shop,Tea Room,Smoke Shop,Donut Shop,Juice Bar,Insurance Office,Business Service,Gym,Cupcake Shop,Cosmetics Shop


In [23]:
khobar_data = khobar_data.rename(columns={"name_en": "District"})
dammam_data = dammam_data.rename(columns={"name_en": "District"})
khobar_data

,District,district_id,center
0,At Tahliyah Dist.,10500031001,"[26.17781863856502, 50.19358769394619]"
1,Ibn Sina Dist.,10500031002,"[26.241028340781252, 50.201773003593736]"
2,Al Hizam Al Akhdar Dist.,10500031003,"[26.305178597317074, 50.20295822975609]"
3,Sinaiyah Ath Thuqbah Dist.,10500031004,"[26.253792497222218, 50.19638499861113]"
4,At Taawun Dist.,10500031005,"[26.226340009404773, 50.18608636999999]"
5,Ar Rakah Al Janubiyah Dist.,10500031006,"[26.350680400337087, 50.203745644269645]"
6,Al Khubar Ash Shamaliyah Dist.,10500031007,"[26.29401644211008, 50.214117997064186]"
7,Madinat Al Ummal Dist.,10500031008,"[26.294228553913037, 50.205489442898546]"
8,Al Aqrabiyah Dist.,10500031009,"[26.297885457971017, 50.189789413188414]"
9,Al Khubar Al Janubiyah Dist.,10500031010,"[26.27328286536233, 50.20615011927536]"


In [24]:
# set number of clusters
kclusters = 5

khobar_grouped_clustering = khobar_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(khobar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

# add clustering labels
khobar_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

khobar_merged = khobar_data

# merge khobar_grouped with khobar_data to add latitude/longitude for each District
khobar_merged = khobar_merged.join(khobar_venues_sorted.set_index('District'), on='District')

khobar_merged.dropna(axis=0, inplace = True)

khobar_merged.head() # check the last columns!

,District,district_id,center,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,At Tahliyah Dist.,10500031001,"[26.17781863856502, 50.19358769394619]",1.0,Soccer Field,Breakfast Spot,Garden,Harbor / Marina,Yoga Studio,Farm,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant
1,Ibn Sina Dist.,10500031002,"[26.241028340781252, 50.201773003593736]",1.0,Auto Workshop,Hookah Bar,Vape Store,Smoke Shop,Shipping Store,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Flower Shop,Fast Food Restaurant
2,Al Hizam Al Akhdar Dist.,10500031003,"[26.305178597317074, 50.20295822975609]",1.0,Coffee Shop,Burger Joint,Juice Bar,Café,Dessert Shop,Hotel,Shawarma Place,Seafood Restaurant,Bakery,Salad Place
3,Sinaiyah Ath Thuqbah Dist.,10500031004,"[26.253792497222218, 50.19638499861113]",1.0,Auto Workshop,Smoke Shop,Motorcycle Shop,Café,Tourist Information Center,Juice Bar,Auto Garage,Yoga Studio,Farmers Market,Food Service
4,At Taawun Dist.,10500031005,"[26.226340009404773, 50.18608636999999]",0.0,Hookah Bar,Spa,Restaurant,Stables,Farmers Market,Café,Auto Garage,Athletics & Sports,Entertainment Service,Ethiopian Restaurant


In [25]:
# set number of clusters
kclusters = 5

dammam_grouped_clustering = dammam_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dammam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

# add clustering labels
dammam_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

dammam_merged = dammam_data

# merge dammam_grouped with dammam_data to add latitude/longitude for each District
dammam_merged = dammam_merged.join(dammam_venues_sorted.set_index('District'), on='District')

dammam_merged.dropna(axis=0, inplace = True)

dammam_merged.head() # check the last columns!

,District,district_id,center,randNumCol,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Khalidiyah Ash Shamaliyah Dist.,10500013001,"[26.413035744059407, 50.14636082148513]",45,0.0,Furniture / Home Store,Yemeni Restaurant,Flea Market,Film Studio,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,Dry Cleaner
1,An Nasriyah Dist.,10500013002,"[26.424016825090902, 50.12204671218182]",83,1.0,Furniture / Home Store,Park,Event Space,Breakfast Spot,Intersection,Cricket Ground,Shopping Mall,Chocolate Shop,Pastry Shop,Costume Shop
2,1St Industrial Dist.,10500013003,"[26.39701435782051, 50.14309465038462]",60,0.0,Furniture / Home Store,Coffee Shop,Construction & Landscaping,Harbor / Marina,Fish & Chips Shop,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store
3,Al Fanar Dist.,10500013004,"[26.40588090352941, 50.19358098470589]",39,1.0,Burger Joint,Fish & Chips Shop,IT Services,Yemeni Restaurant,Film Studio,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store
4,Al Athir Dist.,10500013005,"[26.434276787415733, 50.06036651483147]",10,1.0,Hookah Bar,Juice Bar,Afghan Restaurant,Yemeni Restaurant,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space


In [26]:
khobar_merged["Latitude"] = [ x[0] for x in khobar_merged["center"].tolist() ]
khobar_merged["Longitude"] = [ x[1] for x in khobar_merged["center"].tolist() ]

dammam_merged["Latitude"] = [ x[0] for x in dammam_merged["center"].tolist() ]
dammam_merged["Longitude"] = [ x[1] for x in dammam_merged["center"].tolist() ]

khobar_merged

,District,district_id,center,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,At Tahliyah Dist.,10500031001,"[26.17781863856502, 50.19358769394619]",1.0,Soccer Field,Breakfast Spot,Garden,Harbor / Marina,Yoga Studio,Farm,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,26.177819,50.193588
1,Ibn Sina Dist.,10500031002,"[26.241028340781252, 50.201773003593736]",1.0,Auto Workshop,Hookah Bar,Vape Store,Smoke Shop,Shipping Store,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Flower Shop,Fast Food Restaurant,26.241028,50.201773
2,Al Hizam Al Akhdar Dist.,10500031003,"[26.305178597317074, 50.20295822975609]",1.0,Coffee Shop,Burger Joint,Juice Bar,Café,Dessert Shop,Hotel,Shawarma Place,Seafood Restaurant,Bakery,Salad Place,26.305179,50.202958
3,Sinaiyah Ath Thuqbah Dist.,10500031004,"[26.253792497222218, 50.19638499861113]",1.0,Auto Workshop,Smoke Shop,Motorcycle Shop,Café,Tourist Information Center,Juice Bar,Auto Garage,Yoga Studio,Farmers Market,Food Service,26.253792,50.196385
4,At Taawun Dist.,10500031005,"[26.226340009404773, 50.18608636999999]",0.0,Hookah Bar,Spa,Restaurant,Stables,Farmers Market,Café,Auto Garage,Athletics & Sports,Entertainment Service,Ethiopian Restaurant,26.226340,50.186086
5,Ar Rakah Al Janubiyah Dist.,10500031006,"[26.350680400337087, 50.203745644269645]",1.0,Gym,Coffee Shop,Café,Ice Cream Shop,Restaurant,Flower Shop,Breakfast Spot,Stables,Grocery Store,Photography Studio,26.350680,50.203746
6,Al Khubar Ash Shamaliyah Dist.,10500031007,"[26.29401644211008, 50.214117997064186]",1.0,Coffee Shop,Middle Eastern Restaurant,Café,Tailor Shop,Pakistani Restaurant,Furniture / Home Store,Fried Chicken Joint,Seafood Restaurant,Egyptian Restaurant,Dessert Shop,26.294016,50.214118
7,Madinat Al Ummal Dist.,10500031008,"[26.294228553913037, 50.205489442898546]",1.0,Seafood Restaurant,Coffee Shop,Flower Shop,Gym / Fitness Center,Athletics & Sports,Shawarma Place,Café,Stadium,Clothing Store,Gift Shop,26.294229,50.205489
8,Al Aqrabiyah Dist.,10500031009,"[26.297885457971017, 50.189789413188414]",1.0,Coffee Shop,Shawarma Place,Breakfast Spot,Bakery,Fast Food Restaurant,Tea Room,Indian Restaurant,BBQ Joint,Pastry Shop,Egyptian Restaurant,26.297885,50.189789
9,Al Khubar Al Janubiyah Dist.,10500031010,"[26.27328286536233, 50.20615011927536]",1.0,Bakery,Fried Chicken Joint,Pizza Place,Coffee Shop,Clothing Store,Sandwich Place,Food Truck,Middle Eastern Restaurant,Soccer Field,Market,26.273283,50.206150


In [27]:
# create map
map_clusters = folium.Map(location=[26.2172,50.1971], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(khobar_merged['Latitude'], khobar_merged['Longitude'], khobar_merged['District'], khobar_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# Cluster 0:

khobar_merged.loc[khobar_merged['Cluster Label'] == 0, khobar_merged.columns[[1] + list(range(5, khobar_merged.shape[1]))]]

,district_id,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
4,10500031005,Spa,Restaurant,Stables,Farmers Market,Café,Auto Garage,Athletics & Sports,Entertainment Service,Ethiopian Restaurant,26.226340,50.186086
33,10500031034,Hookah Bar,History Museum,Farm,Yoga Studio,Event Service,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,26.164534,50.148142
36,10500031037,Hookah Bar,Street Art,Yoga Studio,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market,26.183197,50.153691


In [29]:
# Cluster 1

khobar_merged.loc[khobar_merged['Cluster Label'] == 1, khobar_merged.columns[[1] + list(range(5, khobar_merged.shape[1]))]]

,district_id,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,10500031001,Breakfast Spot,Garden,Harbor / Marina,Yoga Studio,Farm,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,26.177819,50.193588
1,10500031002,Hookah Bar,Vape Store,Smoke Shop,Shipping Store,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Flower Shop,Fast Food Restaurant,26.241028,50.201773
2,10500031003,Burger Joint,Juice Bar,Café,Dessert Shop,Hotel,Shawarma Place,Seafood Restaurant,Bakery,Salad Place,26.305179,50.202958
3,10500031004,Smoke Shop,Motorcycle Shop,Café,Tourist Information Center,Juice Bar,Auto Garage,Yoga Studio,Farmers Market,Food Service,26.253792,50.196385
5,10500031006,Coffee Shop,Café,Ice Cream Shop,Restaurant,Flower Shop,Breakfast Spot,Stables,Grocery Store,Photography Studio,26.350680,50.203746
6,10500031007,Middle Eastern Restaurant,Café,Tailor Shop,Pakistani Restaurant,Furniture / Home Store,Fried Chicken Joint,Seafood Restaurant,Egyptian Restaurant,Dessert Shop,26.294016,50.214118
7,10500031008,Coffee Shop,Flower Shop,Gym / Fitness Center,Athletics & Sports,Shawarma Place,Café,Stadium,Clothing Store,Gift Shop,26.294229,50.205489
8,10500031009,Shawarma Place,Breakfast Spot,Bakery,Fast Food Restaurant,Tea Room,Indian Restaurant,BBQ Joint,Pastry Shop,Egyptian Restaurant,26.297885,50.189789
9,10500031010,Fried Chicken Joint,Pizza Place,Coffee Shop,Clothing Store,Sandwich Place,Food Truck,Middle Eastern Restaurant,Soccer Field,Market,26.273283,50.206150
10,10500031011,Movie Theater,Pool,Coffee Shop,Shipping Store,Café,Falafel Restaurant,Food & Drink Shop,Flower Shop,Fast Food Restaurant,26.333200,50.206892


In [30]:
# Cluster 2

khobar_merged.loc[khobar_merged['Cluster Label'] == 2, khobar_merged.columns[[1] + list(range(5, khobar_merged.shape[1]))]]

,district_id,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
37,10500031038,Yoga Studio,Event Service,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market,Farm,26.095232,50.145751


In [31]:
# Cluster 3

khobar_merged.loc[khobar_merged['Cluster Label'] == 3, khobar_merged.columns[[1] + list(range(5, khobar_merged.shape[1]))]]

,district_id,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
34,10500031035,Yoga Studio,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market,26.152318,50.124290
39,10500031040,Yoga Studio,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market,26.143114,50.145837


In [32]:
# Cluster 4

khobar_merged.loc[khobar_merged['Cluster Label'] == 4, khobar_merged.columns[[1] + list(range(5, khobar_merged.shape[1]))]]

,district_id,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
32,10500031033,Trail,Yoga Studio,French Restaurant,Food Truck,Food Service,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market,26.199035,50.171228


In [33]:
# create map
map_clusters = folium.Map(location=[26.2172,50.1971], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dammam_merged['Latitude'], dammam_merged['Longitude'], dammam_merged['District'], dammam_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
dammam_merged.loc[dammam_merged['Cluster Label'] == 3, dammam_merged.columns[[1] + list(range(5, dammam_merged.shape[1]))]]

,district_id,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
21,10500013022,Auto Garage,Palace,Doner Restaurant,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,26.450579,50.018045
42,10500013043,Auto Garage,Auto Workshop,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,Dry Cleaner,26.447342,50.049518
70,10500013072,Auto Garage,Auto Workshop,Café,Doner Restaurant,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,26.403955,50.166384
79,10500013081,Farm,Yemeni Restaurant,Concert Hall,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Dry Cleaner,26.485221,49.933393


to do?